### DATA

In [16]:
from brokers import get_broker
import MetaTrader5 as mt5
import sqlite3
from datetime import datetime
import pandas as pd

class MT5DatabaseSaver:
    def __init__(self, broker_name, db_path="market_data.db"):
        self.broker_name = broker_name.lower()
        self.db_path = db_path

        # Initialize broker
        self.broker = get_broker(broker_name)
        self.broker.initialize()

        # SQLite connection
        self.conn = sqlite3.connect(self.db_path)
        self.cursor = self.conn.cursor()

        # Create broker-specific tables
        self._create_tables()

    def _table(self, base_name):
        return f"{self.broker_name}_{base_name}"

    def _create_tables(self):
        self.cursor.execute(f"""
            CREATE TABLE IF NOT EXISTS {self._table("market_data")} (
                symbol TEXT NOT NULL,
                timeframe TEXT NOT NULL,
                timestamp TEXT NOT NULL,
                open REAL,
                high REAL,
                low REAL,
                close REAL,
                tick_volume INTEGER,
                PRIMARY KEY (symbol, timeframe, timestamp)
            )
        """)

        self.cursor.execute(f"""
            CREATE TABLE IF NOT EXISTS {self._table("symbol_metadata")} (
                symbol TEXT PRIMARY KEY,
                minimal_volume REAL,
                volume_step REAL,
                margin_currency TEXT,
                profit_currency TEXT
            )
        """)
        self.conn.commit()

    def save_data(self, symbol, timeframe, rates):
        for rate in rates:
            time = datetime.utcfromtimestamp(int(rate['time'])).strftime("%Y-%m-%d %H:%M:%S")

            self.cursor.execute(f"""
                INSERT OR IGNORE INTO {self._table("market_data")}
                (symbol, timeframe, timestamp, open, high, low, close, tick_volume)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?)
            """, (
                str(symbol),
                str(timeframe),
                time,
                float(rate['open']),
                float(rate['high']),
                float(rate['low']),
                float(rate['close']),
                int(rate['tick_volume'])
            ))
        self.conn.commit()

    def update_db(self, symbol="EURUSD", timeframe=mt5.TIMEFRAME_M1, num_bars=10):
        if not mt5.initialize():
            raise RuntimeError("MT5 initialization failed")

        rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, num_bars)
        if rates is None or len(rates) == 0:
            mt5.shutdown()
            raise ValueError(f"No data returned for {symbol} on timeframe {timeframe}")

        closed_rates = rates[:-1]  # drop last forming candle

        self.save_data(symbol, timeframe, closed_rates)
        self.save_symbol_info(symbol)

        mt5.shutdown()

    def save_symbol_info(self, symbol="EURUSD"):
        if not mt5.initialize():
            raise RuntimeError("MT5 initialization failed")

        info = mt5.symbol_info(symbol)
        if info is None:
            mt5.shutdown()
            raise ValueError(f"Symbol {symbol} not found")

        self.cursor.execute(f"""
            INSERT OR REPLACE INTO {self._table("symbol_metadata")}
            (symbol, minimal_volume, volume_step, margin_currency, profit_currency)
            VALUES (?, ?, ?, ?, ?)
        """, (
            info.name,
            float(info.volume_min),
            float(info.volume_step),
            getattr(info, "currency_margin", None),
            getattr(info, "currency_profit", None)
        ))
        self.conn.commit()

        mt5.shutdown()

    def load_ohlcv(self, symbol="EURUSD", timeframe="1", limit=500):
        """Load OHLCV data for this broker instance."""
        table_name = self._table("market_data")
        query = f"""
            SELECT timestamp, open, high, low, close, tick_volume
            FROM {table_name}
            WHERE symbol = ? AND timeframe = ?
            ORDER BY timestamp ASC
            LIMIT ?
        """

        df = pd.read_sql_query(query, self.conn, params=(symbol, timeframe, limit))

        df.rename(columns={"tick_volume": "volume"}, inplace=True)
        df["timestamp"] = pd.to_datetime(df["timestamp"], utc=True)
        df.set_index("timestamp", inplace=True)

        return df
        
    def load_symbol_info(self, symbol="EURUSD"):
        """Load symbol metadata for this broker instance."""
        table_name = self._table("symbol_metadata")
        query = f"""
            SELECT *
            FROM {table_name}
            WHERE symbol = ?
        """

        df = pd.read_sql_query(query, self.conn, params=(symbol,))
        if df.empty:
            return None
        return df.iloc[0].to_dict()

    def close(self):
        self.conn.close()

In [45]:
import MetaTrader5 as mt5
from data import MT5DatabaseSaver

acg_data = MT5DatabaseSaver("acg")

# Fetch and store last 100 M1 candles
acg_data.update_db(symbol="USOIL.pro", timeframe=mt5.TIMEFRAME_M30, num_bars=99999)

df = acg_data.load_ohlcv(symbol="USOIL.pro", timeframe="30", limit=200000)
print(df.tail())
eurusd_data = acg_data.load_symbol_info(symbol="USOIL.pro")
print(eurusd_data)
acg_data.close()



                             open    high     low   close  volume
timestamp                                                        
2026-01-29 04:30:00+00:00  63.665  63.932  63.602  63.848    1200
2026-01-29 05:00:00+00:00  63.852  63.921  63.781  63.823    1654
2026-01-29 05:30:00+00:00  63.823  63.928  63.712  63.866     808
2026-01-29 06:00:00+00:00  63.865  64.163  63.841  64.103     664
2026-01-29 06:30:00+00:00  64.101  64.184  63.966  64.080     837
{'symbol': 'USOIL.pro', 'minimal_volume': 0.01, 'volume_step': 0.01, 'margin_currency': 'USD', 'profit_currency': 'USD'}


In [46]:
df

,open,high,low,close,volume
timestamp,,,,,
2022-03-31 12:00:00+00:00,101.509,101.762,100.703,100.993,3546
2022-03-31 12:30:00+00:00,100.989,101.566,100.406,100.711,2920
2022-03-31 13:00:00+00:00,100.702,100.898,100.202,100.538,4172
2022-03-31 13:30:00+00:00,100.551,100.767,99.525,100.342,3421
2022-03-31 14:00:00+00:00,100.350,101.312,100.133,101.265,3762
...,...,...,...,...,...
2026-01-29 04:30:00+00:00,63.665,63.932,63.602,63.848,1200
2026-01-29 05:00:00+00:00,63.852,63.921,63.781,63.823,1654
2026-01-29 05:30:00+00:00,63.823,63.928,63.712,63.866,808


### SIGNAL

In [47]:
from MLmodel import evaluate_symbol



report, label_stats, model = evaluate_symbol(df, stddev=0.5)

results = []       
results.append({
    "symbol": df,
    "report": report,
    "label_stats": label_stats
})
print(results)



threshold: 0.17%
Raw class weights: {0: 0.5720955966695006, -1: 1.573294737623395, 1: 1.622240079592852}
[{'symbol':                               open     high      low    close  volume
timestamp                                                            
2022-03-31 12:00:00+00:00  101.509  101.762  100.703  100.993    3546
2022-03-31 12:30:00+00:00  100.989  101.566  100.406  100.711    2920
2022-03-31 13:00:00+00:00  100.702  100.898  100.202  100.538    4172
2022-03-31 13:30:00+00:00  100.551  100.767   99.525  100.342    3421
2022-03-31 14:00:00+00:00  100.350  101.312  100.133  101.265    3762
...                            ...      ...      ...      ...     ...
2026-01-29 04:30:00+00:00   63.665   63.932   63.602   63.848    1200
2026-01-29 05:00:00+00:00   63.852   63.921   63.781   63.823    1654
2026-01-29 05:30:00+00:00   63.823   63.928   63.712   63.866     808
2026-01-29 06:00:00+00:00   63.865   64.163   63.841   64.103     664
2026-01-29 06:30:00+00:00   64.101   64.184

In [4]:
from MLmodel import generate_features
import xgboost as xgb
import MetaTrader5 as mt5
from data import MT5DatabaseSaver

acg_data = MT5DatabaseSaver("icmarkets")
acg_data.update_db(symbol="EURUSD", timeframe=mt5.TIMEFRAME_M1, num_bars=300)
df_live = acg_data.load_ohlcv(symbol="EURUSD", timeframe="1", limit=300)

model = xgb.XGBClassifier()
model.load_model("xgb_multiclass.json")


features = generate_features(df_live)

X_live = features.iloc[-1:].values  # last row only
prediction = model.predict(X_live)

print(prediction)
print(df_live.tail(1))
# --- IGNORE ---

[1]
                            open     high      low  close  volume
timestamp                                                        
2026-01-29 06:59:00+00:00  1.199  1.19902  1.19897  1.199      31


In [9]:
df_live

,open,high,low,close,volume
timestamp,,,,,
2026-01-29 01:58:00+00:00,1.19767,1.19773,1.19766,1.19768,57
2026-01-29 01:59:00+00:00,1.19769,1.19809,1.19763,1.19809,37
2026-01-29 02:00:00+00:00,1.19806,1.19806,1.19754,1.19756,121
2026-01-29 02:01:00+00:00,1.19751,1.19762,1.19712,1.19722,106
2026-01-29 02:02:00+00:00,1.19722,1.19733,1.19703,1.19715,109
...,...,...,...,...,...
2026-01-29 06:53:00+00:00,1.19898,1.19906,1.19893,1.19893,46
2026-01-29 06:54:00+00:00,1.19893,1.19907,1.19893,1.19901,47
2026-01-29 06:55:00+00:00,1.19901,1.19901,1.19880,1.19881,42


### BACKTEST

### RISK

### EXECUTION

### POTFOLIO ANALYTICS